# Ephemeris Experiments
Sam Greydanus | 2023
### Setup
Before running this code, you'll need to download the raw ephemeris data for the planets (or other celestial bodies) that you want to plot. To do this, go to [https://ssd.jpl.nasa.gov/horizons/app.html#/](https://ssd.jpl.nasa.gov/horizons/app.html#/). For 'Ephemeris Type' select 'Vector Table'. For 'Target Body' select the body you want (eg. 'Earth'). For Coordinate Center try using 'Solar System Barycenter (SSB) '. For time specification, I manually selected a timespan of five years; the default data interval is 1440 minutes (1 day). Once you've chosen your desired settings, click 'Generate Ephemeris' and then click 'Download Results' when the results load. This will let you download a .txt file to a local directory. Name the file after the planet, eg `earth.txt`. Repeat this process for all the planets you want, saving each of them to a different text file in the same folder. Once you have all the .txt files saved to that folder, you will be ready to run this code.

In [ ]:
import numpy as np
import torch, time

import pandas as pd
from functools import partial
import matplotlib.pyplot as plt

def load_planet(planet_name, data_dir):
    '''Reads a file named, eg.,"earth.txt" with ephemeris data in a vector table format
       downloaded from https://ssd.jpl.nasa.gov/horizons/app.html#/'''
    with open(data_dir + '{}.txt'.format(planet_name), 'r') as f:
        text = f.read()

    main_data = text[text.find('$$SOE')+5:text.find('$$EOE')].split('\n')
    s_xyz = main_data[2::4]

    f_xyz = []
    for l in s_xyz:
        splits = [s.strip(' ').split(' ')[0] for s in l.split('=')[1:]]
        f_xyz.append([float(s)*1e3 for s in splits]) # convert from km to meters
    return np.asarray(f_xyz)

def get_colnames(names):
    '''Generates DataFrame column names for each x, y, z coordinate dimension'''
    colnames = []
    for n in names:
        colnames += [n + '_x', n + '_y', n + '_z']
    return colnames

def get_colformat(coords):
    '''Reshape from [planets, time, xyz] to [time, planets*xyz]'''
    N = coords.shape[0]
    return coords.transpose(1,0,2).reshape(-1,N*3)

def process_raw_ephemeris(planets, data_dir, last_n_days=None):
    '''Loads raw ephemeris files for a list of planet names, organizes the data in a DataFrame,
       and then saves the DataFrame as a csv in the same directory as the raw files.'''
    coords = np.stack([load_planet(p, data_dir) for p in planets])
    if last_n_days is not None:
        coords = coords[:,-last_n_days:]
    assert coords.shape[1] > 300, 'length should be over 300'
    coords = np.concatenate([coords[:,:150], coords[:,-150:]], axis=1)
    df = pd.DataFrame(data=get_colformat(coords), columns=get_colnames(planets))
    df.to_csv(data_dir + 'ephemeris_ablate.csv')
    return df

In [ ]:
def get_colors():
    return {'sun':'yellow','venus':'orange','mercury':'pink','earth':'blue','mars':'red'}

def plot_planets(df, planets, fig=None):
    colors = get_colors()
    fig = plt.figure(figsize=[5,5], dpi=100) if fig is None else fig
    
    for i, name in enumerate(planets):
        x, y, z = df[name + '_x'], df[name + '_y'], df[name + '_z']
        plt.plot(x, y, '.', alpha=0.33, color=colors[name], label=name + ' (data)', markersize=2)
        #plt.plot(x.iloc[0], y.iloc[0], 'x', alpha=0.33, color=colors[name])
        plt.plot(x.iloc[-1], y.iloc[-1], '.', alpha=0.33, color=colors[name], markersize=9)
    plt.title("Ephemeris data from JPL's Horizon System")
    plt.tight_layout() ; plt.axis('equal')
    return fig
    
# planets = ['sun', 'venus', 'earth', 'mars','mercury']
planets = ['sun', 'mercury', 'venus', 'earth', 'mars']
data_dir = './data/'
df = process_raw_ephemeris(planets, data_dir, last_n_days=365) #365
plot_planets(df, planets)
plt.legend(fontsize=7,  loc='upper right') ; plt.show()

## Make coplanar

In [ ]:
# e = df[['earth_x', 'earth_y', 'earth_z']]
m = df[['mars_x', 'mars_y', 'mars_z']]
# s = df[['sun_x', 'sun_y', 'sun_z']]
# v = df[['venus_x', 'venus_y', 'venus_z']]
# m = df[['mercury_x', 'mercury_y', 'mercury_z']]
plt.plot(m)

### It turns out that all the orbits are more or less coplanar already

## Simulate the planet trajectories & compare to data

In [ ]:
def V_planets(xs, masses, eps=1e-10, G=6.67e-11): # # 2e-25
    if len(xs.shape) > 2:
        return sum([V_planets(_xs, masses, eps=eps, G=G) for _xs in xs]) # broadcast
    else:
        ixs = torch.triu_indices(xs.shape[0], xs.shape[0], 1).split(1)
        dist_matrix = ((xs[:,0:1] - xs[:,0:1].T).pow(2) + (xs[:,1:2] - xs[:,1:2].T).pow(2) + eps).sqrt()
        mM_matrix = torch.tensor( masses[None,:] * masses[:,None] )
        U_vals = G * mM_matrix[ixs] / dist_matrix[ixs]
        return -U_vals.sum()
    
def accelerations(xs, masses, potential_fn, **kwargs):
    xs.requires_grad = True
    forces = -torch.autograd.grad(potential_fn(xs), xs)[0]
    return forces/masses[:,None]
    
def solve_ode_euler(x0, x1, dt, accel_fn, steps=100, box_width=1):
    xs = [x0, x1]
    ts = [0, dt]
    v = (x1 - x0) / dt
    x = xs[-1]
    for i in range(steps-2):
        a = accel_fn(x)
        v = v + a*dt
        x = x + v*dt
        xs.append(x)
        ts.append(ts[-1]+dt)
    return np.asarray(ts), np.stack(xs)

In [ ]:
def get_coords(df, planets, i=0):
    return np.asarray([ [df[p + '_x'].iloc[i], df[p + '_y'].iloc[i]] for p in planets])

def get_masses(planets):
    d = {'sun':1.99e30, 'venus':4.87e24, 'mercury':3.3e23, 'earth':5.97e24, 'mars':6.42e23}
    return np.asarray([d[p] for p in planets])

def simulate_planets(dt=24*60*60, steps=365-300):
    x0 = get_coords(df, planets, i=148)
    x1 = get_coords(df, planets, i=149)
    V_planets_fn = partial(V_planets, masses=get_masses(planets))
    accel_fn = lambda x: accelerations(torch.tensor(x), get_masses(planets), V_planets_fn).numpy()
    return solve_ode_euler(x0, x1, dt, accel_fn, steps=365-300)

t_sim, x_sim = simulate_planets()

In [ ]:
plot_planets(df, planets)

colors = get_colors()
for i, (planet, coords) in enumerate(zip(planets, x_sim.transpose(1,2,0))):
    x, y = coords
    plt.plot(x, y, ':', alpha=0.5, color=colors[planet], label=planets[i] + ' (sim)')
    plt.plot(x[0], y[0], '+', color=colors[planet])
    plt.plot(x[-1], y[-1], 'x', color=colors[planet])
plt.axis('equal')

plt.legend(fontsize=6,  loc='upper right', ncol=2) ; plt.show()

## Simulate by minimizing the action

In [ ]:
def lagrangian_planets(x, xdot, masses):
    N = x.shape[-1] // 2
    norm_factor = x.shape[0]*N
    xdot = xdot.reshape(-1,N,2)
    m = torch.tensor(masses[None,:,None])
    T = (.5*m*xdot**2).sum()
    V = V_planets(x.reshape(-1, N, 2), masses).sum()
    return (T - V) / norm_factor
  
def action(x, L_fn, dt):
    xdot = (x[1:] - x[:-1]) / dt
    xdot = torch.cat([xdot, xdot[-1:]], axis=0)
    return L_fn(x, xdot).sum()

def get_path_between(path, steps, step_size, L_fn, dt, opt='sgd', print_every=15):
    t = np.linspace(0, len(path.x)-1, len(path.x)) * dt
    optimizer = torch.optim.SGD(path.parameters(), lr=step_size, momentum=0) if opt=='sgd' else \
                torch.optim.Adam(path.parameters(), lr=step_size)
    xs = [path.x.clone().data]
    t0 = time.time()
    for i in range(steps):
        S = action(path.x, L_fn, dt)
        S.backward() ; path.x.grad.data[[0,-1]] *= 0
        optimizer.step() ; path.zero_grad()

        if i % (steps//print_every) == 0:
            xs.append(path.x.clone().detach().data)
            print('step={:04d}, S={:.3e} J*s, dt={:.1f}s'.format(i, S.item(), time.time()-t0))
            t0 = time.time()
    return t, path, xs

class PerturbedPath(torch.nn.Module):
    def __init__(self, x_true, N, sigma=0, shift=False, zero_basepath=False, coords=2):
        super(PerturbedPath, self).__init__()
        np.random.seed(0)
        self.x_true = x_true
        x_noise = sigma*np.random.randn(*x_true.shape).clip(-1,1)
        x_noise[:1] = x_noise[-1:] = 0
        x_noise[:,0,:] = 0 # don't perturb the Sun
        x_basepath = np.copy(x_true)
        x_basepath[1:-1] = x_basepath[1:-1]*0 if zero_basepath else x_basepath[1:-1]
        self.x_pert = x_pert = (x_basepath + x_noise).reshape(-1, N*coords)
        if shift:
            x_pert_shift = np.concatenate([x_pert[5:-5,2:], x_pert[5:-5,:2]], axis=-1)
            self.x_pert[5:-5] = x_pert[5:-5] = x_pert_shift
            print(self.x_pert.shape)
        self.x = torch.nn.Parameter(torch.tensor(x_pert)) # [time, N*2]

In [ ]:
dt = 24*60*60 ; N = len(planets)

t_sim, x_sim = simulate_planets(dt=dt)
init_path = PerturbedPath(x_sim, N=N, sigma=2e10) # [time, N*2]

L_planets = partial(lagrangian_planets, masses=get_masses(planets))

t_min, path, xs_min = get_path_between(init_path, steps=500, step_size=1e9,
                                       L_fn=L_planets, dt=dt, opt='adam')

In [ ]:
plt.figure(figsize=[5,3], dpi=120)
plt.title('Earth x coordinate')
xs_sim = init_path.x_true
xs_init = xs_min[0].detach().numpy().reshape(-1,N,2)
xs_final = xs_min[-1].detach().numpy().reshape(-1,N,2)
plt.plot(xs_sim[:,2,0], '--', label='sim')
plt.plot(xs_init[:,2,0], alpha=0.5, label='init')
plt.plot(xs_final[:,2,0], alpha=0.5, label='final')
plt.legend()

In [ ]:
fig = plt.figure(figsize=[5,5], dpi=140)
plot_planets(df, planets, fig=fig)
colors = get_colors()

xs = xs_min[0].detach().numpy().reshape(-1,N,2)
for i, (planet, coords) in enumerate(zip(planets, xs.transpose(1,2,0))):
    x, y = coords
    plt.plot(x, y, '.', alpha=0.3, color=colors[planet], label=planets[i] + ' (init)')
    plt.plot(x[0], y[0], '+', color=colors[planet])
    plt.plot(x[-1], y[-1], 'x', color=colors[planet])
    
xs = xs_min[-1].detach().numpy().reshape(-1,N,2)
for i, (planet, coords) in enumerate(zip(planets, xs.transpose(1,2,0))):
    x, y = coords
    plt.plot(x, y, ':', alpha=0.5, color=colors[planet], label=planets[i] + ' (path)')

plt.axis('equal')
plt.legend(fontsize=6,  loc='upper right', ncol=3) ; plt.show()

In [ ]:
# FORCES

# sun-venus 5.60e22 # 5.47e22
# sun-earth 3.63e22 # 3.52e22
# sun-mars 1.75e21 # 1.65e21 
# sun-mercury 9.80e21 # 9.3e21
# venus-earth 4.50e17
# venus-mars 3.5e15
# venus-mercury 1.5e16
# earth-mars 2.67e15
# earth-mercury 6.1e15
# mars-mercury 4.76e14

In [ ]:
# m = np.asarray([get_mass(p) for p in planets])
# m

In [ ]:
# def simulate_planets(dt=0.5, steps=100):
#     np.random.seed(1)
#     x0 = np.asarray([[0.4, 0.3], [0.4, 0.7], [0.7, 0.5]])
#     v0 = np.asarray([[0.017, -0.006], [-.012, -.012], [0.0, 0.017]])
#     x1 = x0 + dt*v0
#     forces_fn = lambda x: forces(torch.tensor(x), V_planets).numpy()
#     return solve_ode_euler(x0, x1, dt, forces_fn, steps=steps)